In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

def extract_tables_from_mops(year, season):
    # 初始化 Chrome 無痕視窗
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")# 無痕模式（Incognito Mode）
    options.add_argument("--no-sandbox") # 關閉沙箱模式（No Sandbox）在部分 Linux 或 Docker 環境中執行 Chrome 時可能無法啟動
    options.add_argument("--disable-gpu")  #停用 GPU 硬體加速
    options.add_argument("--disable-infobars") #關閉「Chrome 正受自動測試軟體控制」的提示條
    options.add_argument("--start-maximized") # 啟動時最大化視窗    options = webdriver.ChromeOptions()
    #options.add_argument("--headless")  # 可選：不開啟視窗 
    #options.add_argument("--headless=new")  # 如需無頭模式可啟用
    driver = webdriver.Chrome(options=options)

    url = "https://mops.twse.com.tw/mops/#/web/t163sb04"
    driver.get(url)

#    try:
    # 選擇「上市」
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "TYPEK")))
    driver.find_element(By.ID, "TYPEK").find_element(By.XPATH, "//option[@value='otc']").click() #sii上市 otc上櫃
    time.sleep(1)

    # 輸入民國年
    driver.find_element(By.ID, "year").clear()
    driver.find_element(By.ID, "year").send_keys(str(year))
    time.sleep(1)

    # 選擇季別
    driver.find_element(By.ID, "season").find_element(By.XPATH, f"//option[@value='{int(season):02}']").click()
    time.sleep(1)

    # 點擊查詢
    driver.find_element(By.ID, "searchBtn").click()
    time.sleep(1)

    # 切換到新視窗
    WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
    driver.switch_to.window(driver.window_handles[-1])
    print("🔀 已切換到新視窗")

    # 等待表格出現
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "hasBorder"))
    )

    soup = BeautifulSoup(driver.page_source, "html.parser")
    tables = soup.find_all("table", class_="hasBorder")

    

    if len(tables) >= 2:
        table = tables[1]  # 第 3 個表格（index 從 0 開始）
        
        df = pd.read_html(str(table))[0]  # ✅ 直接抓完整表格（含第一列為欄位名）

        print("\n📄 第 2 個表格內容：")
        print(df.head())
    
    print(df.head())
    return df




if __name__ == "__main__":
    for year in range(105, 114):  # 民國104 ~ 113年
        for season in range(1, 5):  # 第1~4季
            print(f"\n📦 正在抓取：{year} 年 第 {season} 季")
            try:
                df = extract_tables_from_mops(year, season)
                filename = f"MOPS_第2表格otc_{year}Q{season}.csv"
                df.to_csv(filename, index=False, encoding="utf-8-sig")
                print(f"💾 已儲存 {filename}")
            except Exception as e:
                print(f"❌ 抓取失敗：{year}Q{season} ➜ {e}")
            time.sleep(3)  # 加一點延遲避免被擋





📦 正在抓取：105 年 第 1 季
🔀 已切換到新視窗


C:\Users\Robby1206\AppData\Local\Temp\ipykernel_20464\1569839851.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # ✅ 直接抓完整表格（含第一列為欄位名）



📄 第 2 個表格內容：
   公司 代號 公司名稱     營業收入    營業成本 營業毛利（毛損） 未實現銷貨（損）益 已實現銷貨（損）益 營業毛利（毛損）淨額  \
0   1259   安心  1112922  818919   294003        --        --     294003   
1   1264   德麥   941946  587930   354016        18        18     354016   
2   1336   台翰   446486  416402    30084        --        --      30084   
3   1565   精華  1512982  879268   633714        --        --     633714   
4   1569   濱川   495092  439526    55566        --        --      55566   

     營業費用 其他收益及費損淨額  ...  其他綜合損益（淨額）  合併前非屬共同控制股權綜合損益淨額  本期綜合損益總額  \
0  263477        --  ...        -491                 --     25488   
1  194759        --  ...        -983                 --    120914   
2   59894        --  ...      -22844                 --    -50300   
3  113337        --  ...       -1796                 --    406875   
4   66360        --  ...      -35463                 --    -55360   

  淨利（淨損）歸屬於母公司業主  淨利（淨損）歸屬於共同控制下前手權益 淨利（淨損）歸屬於非控制權益 綜合損益總額歸屬於母公司業主  \
0          33061                  --          -7082     

C:\Users\Robby1206\AppData\Local\Temp\ipykernel_20464\1569839851.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # ✅ 直接抓完整表格（含第一列為欄位名）



📄 第 2 個表格內容：
   公司 代號  公司名稱     營業收入     營業成本 營業毛利（毛損） 未實現銷貨（損）益 已實現銷貨（損）益 營業毛利（毛損）淨額  \
0   1259    安心  2302667  1674900   627767        --        --     627767   
1   1264    德麥  1849262  1161052   688210        10        18     688218   
2   1268  漢來美食  1607421   912862   694559        --        --     694559   
3   1336    台翰   800935   779021    21914        --        --      21914   
4   1565    精華  3060446  1753215  1307231        --        --    1307231   

     營業費用 其他收益及費損淨額  ...  其他綜合損益（淨額）  合併前非屬共同控制股權綜合損益淨額  本期綜合損益總額  \
0  530179        --  ...       -1858                 --     83634   
1  383557        --  ...      -15209                 --    224380   
2  543019        --  ...       17499                 --    165850   
3  121327        --  ...      -35386                 --   -121469   
4  232556        --  ...       -1539                 --    847166   

  淨利（淨損）歸屬於母公司業主  淨利（淨損）歸屬於共同控制下前手權益 淨利（淨損）歸屬於非控制權益 綜合損益總額歸屬於母公司業主  \
0          98328                  --        

...


📄 第 2 個表格內容：
   公司 代號  公司名稱     營業收入     營業成本 原始認列生物資產及農產品之利益（損失） 生物資產當期公允價值減出售成本之變動利益（損失）  \
0   1240  茂生農經  1432034  1267539                7520                    21975   
1   1259    安心  3009628  2317946                  --                       --   
2   1264    德麥  3140695  2124849                  --                       --   
3   1268  漢來美食  2995315  1601201                  --                       --   
4   1294  漢田生技   396944   234559                  --                       --   

  營業毛利（毛損） 未實現銷貨（損）益 已實現銷貨（損）益 營業毛利（毛損）淨額  ...  其他綜合損益（淨額） 合併前非屬共同控制股權綜合損益淨額  \
0   193990        --        --     193990  ...      256763                --   
1   691682        --        --     691682  ...      121123                --   
2  1015846        --        --    1015846  ...       57905                --   
3  1394114        --        --    1394114  ...       37415                --   
4   162385        --        --     162385  ...          --                --   

   本期綜合損益總額  淨利（淨損

C:\Users\Robby1206\AppData\Local\Temp\ipykernel_20464\1569839851.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # ✅ 直接抓完整表格（含第一列為欄位名）



📄 第 2 個表格內容：
   公司 代號  公司名稱     營業收入     營業成本 原始認列生物資產及農產品之利益（損失） 生物資產當期公允價值減出售成本之變動利益（損失）  \
0   1240  茂生農經  2132889  1888849               10836                    43266   
1   1259    安心  4571195  3528302                  --                       --   
2   1264    德麥  4924106  3336429                  --                       --   
3   1268  漢來美食  4426194  2408706                  --                       --   
4   1294  漢田生技   595387   350186                  --                       --   

   營業毛利（毛損） 未實現銷貨（損）益 已實現銷貨（損）益  營業毛利（毛損）淨額  ...  其他綜合損益（淨額）  \
0    298142        --        --      298142  ...      456078   
1   1042893        --        --     1042893  ...       57413   
2   1587677        --        --     1587677  ...       99487   
3   2017488        --        --     2017488  ...       19330   
4    245201        --        --      245201  ...          --   

  合併前非屬共同控制股權綜合損益淨額  本期綜合損益總額  淨利（淨損）歸屬於母公司業主  淨利（淨損）歸屬於共同控制下前手權益  \
0                --    570394          127102

C:\Users\Robby1206\AppData\Local\Temp\ipykernel_20464\1569839851.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # ✅ 直接抓完整表格（含第一列為欄位名）



📄 第 2 個表格內容：
   公司 代號  公司名稱     營業收入     營業成本 原始認列生物資產及農產品之利益（損失） 生物資產當期公允價值減出售成本之變動利益（損失）  \
0   1240  茂生農經  2856900  2531754               22084                    51037   
1   1259    安心  6045581  4683358                  --                       --   
2   1264    德麥  6806342  4603477                  --                       --   
3   1268  漢來美食  5929814  3215909                  --                       --   
4   1294  漢田生技   748423   442680                  --                       --   

   營業毛利（毛損） 未實現銷貨（損）益 已實現銷貨（損）益  營業毛利（毛損）淨額  ...  其他綜合損益（淨額）  \
0    398267        --        --      398267  ...        -687   
1   1362223        --        --     1362223  ...      118194   
2   2202865        --        --     2202865  ...       88982   
3   2713905        --        --     2713905  ...      -14202   
4    305743        --        --      305743  ...          --   

  合併前非屬共同控制股權綜合損益淨額  本期綜合損益總額  淨利（淨損）歸屬於母公司業主  淨利（淨損）歸屬於共同控制下前手權益  \
0                --    165776          179603

In [2]:
import os
import re

def rename_files_to_ad(folder_path):
    # 取得該資料夾下所有檔案
    for filename in os.listdir(folder_path):
        match = re.match(r"MOPS_第2表格otc_(\d{3})Q(\d).csv", filename)
        if match:
            minguo_year = int(match.group(1))  # 104, 105...
            season = match.group(2)            # 1, 2, 3, 4
            ad_year = minguo_year + 1911

            new_filename = f"MOPS_第2表格otc_{ad_year}Q{season}.csv"
            old_path = os.path.join(folder_path, filename)
            new_path = os.path.join(folder_path, new_filename)

            os.rename(old_path, new_path)
            print(f"✅ 已重新命名：{filename} ➜ {new_filename}")

if __name__ == "__main__":
    folder = "./"  # ← 替換為你的資料夾路徑（例如 r"C:\Users\你\Downloads"）
    rename_files_to_ad(folder)


✅ 已重新命名：MOPS_第2表格otc_105Q1.csv ➜ MOPS_第2表格otc_2016Q1.csv
✅ 已重新命名：MOPS_第2表格otc_105Q2.csv ➜ MOPS_第2表格otc_2016Q2.csv
✅ 已重新命名：MOPS_第2表格otc_105Q3.csv ➜ MOPS_第2表格otc_2016Q3.csv
✅ 已重新命名：MOPS_第2表格otc_105Q4.csv ➜ MOPS_第2表格otc_2016Q4.csv
✅ 已重新命名：MOPS_第2表格otc_106Q1.csv ➜ MOPS_第2表格otc_2017Q1.csv
✅ 已重新命名：MOPS_第2表格otc_106Q2.csv ➜ MOPS_第2表格otc_2017Q2.csv
✅ 已重新命名：MOPS_第2表格otc_106Q3.csv ➜ MOPS_第2表格otc_2017Q3.csv
✅ 已重新命名：MOPS_第2表格otc_106Q4.csv ➜ MOPS_第2表格otc_2017Q4.csv
✅ 已重新命名：MOPS_第2表格otc_107Q1.csv ➜ MOPS_第2表格otc_2018Q1.csv
✅ 已重新命名：MOPS_第2表格otc_107Q2.csv ➜ MOPS_第2表格otc_2018Q2.csv
✅ 已重新命名：MOPS_第2表格otc_107Q3.csv ➜ MOPS_第2表格otc_2018Q3.csv
✅ 已重新命名：MOPS_第2表格otc_107Q4.csv ➜ MOPS_第2表格otc_2018Q4.csv
✅ 已重新命名：MOPS_第2表格otc_108Q1.csv ➜ MOPS_第2表格otc_2019Q1.csv
✅ 已重新命名：MOPS_第2表格otc_108Q2.csv ➜ MOPS_第2表格otc_2019Q2.csv
✅ 已重新命名：MOPS_第2表格otc_108Q3.csv ➜ MOPS_第2表格otc_2019Q3.csv
✅ 已重新命名：MOPS_第2表格otc_108Q4.csv ➜ MOPS_第2表格otc_2019Q4.csv
✅ 已重新命名：MOPS_第2表格otc_109Q1.csv ➜ MOPS_第2表格otc_2020Q1.csv
✅ 已重新命名：MOPS_第2表格otc_109Q2.csv 

In [3]:
import pandas as pd
import os

# ✔️ 欲保留欄位（原始欄位名，未清洗前）
columns_to_keep = [
    "公司代號",
    "公司名稱",
    "營業收入",
    "營業毛利（毛損）",
    "營業利益（損失）",
    "營業外收入及支出",
    "稅前淨利（淨損）",
    "基本每股盈餘（元）"
]

def clean_and_add_ratios(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv") and filename.startswith("MOPS_第2表格otc_"):
            file_path = os.path.join(folder_path, filename)
            print(f"\n📄 處理中：{filename}")

            # 讀取檔案
            df = pd.read_csv(file_path, dtype=str)

            # ✅ 清洗欄位名稱：移除空格與逗號
            df.columns = df.columns.str.strip().str.replace(" ", "", regex=False).str.replace(",", "", regex=False)

            # ✅ 清洗 columns_to_keep 使其與 df.columns 對齊
            cleaned_keep = [col.replace(" ", "").replace(",", "") for col in columns_to_keep]

            # ✅ 確認欄位是否存在
            missing = [col for col in cleaned_keep if col not in df.columns]
            if missing:
                print(f"⚠ 缺少欄位：{missing}，跳過")
                continue

            # ✅ 只保留需要的欄位
            df = df[cleaned_keep].copy()

            # ✅ 數值欄位轉換（跳過公司代號與公司名稱）
            for col in cleaned_keep[2:]:
                df[col] = pd.to_numeric(df[col].str.replace(",", ""), errors="coerce")

            # ✅ 新增比率欄位
            df["毛利率"] = (df["營業毛利（毛損）"] / df["營業收入"]).round(4)
            df["營益率"] = (df["營業利益（損失）"] / df["營業收入"]).round(4)
            df["業外稅前率"] = (df["營業外收入及支出"] / df["稅前淨利（淨損）"]).round(4)

            # ✅ 加入年季欄位（來自檔名）
            season_info = filename.replace("MOPS_第2表格otc_", "").replace(".csv", "")
            df["年季"] = season_info

            # ✅ 存回原檔（覆蓋）
            df.to_csv(file_path, index=False, encoding="utf-8-sig")
            print(f"✅ 已更新並儲存：{filename}")

if __name__ == "__main__":
    folder = "./"  # 或指定資料夾
    clean_and_add_ratios(folder)



📄 處理中：MOPS_第2表格otc_2016Q1.csv
✅ 已更新並儲存：MOPS_第2表格otc_2016Q1.csv

📄 處理中：MOPS_第2表格otc_2016Q2.csv
✅ 已更新並儲存：MOPS_第2表格otc_2016Q2.csv

📄 處理中：MOPS_第2表格otc_2016Q3.csv
✅ 已更新並儲存：MOPS_第2表格otc_2016Q3.csv

📄 處理中：MOPS_第2表格otc_2016Q4.csv
✅ 已更新並儲存：MOPS_第2表格otc_2016Q4.csv

📄 處理中：MOPS_第2表格otc_2017Q1.csv
✅ 已更新並儲存：MOPS_第2表格otc_2017Q1.csv

📄 處理中：MOPS_第2表格otc_2017Q2.csv
✅ 已更新並儲存：MOPS_第2表格otc_2017Q2.csv

📄 處理中：MOPS_第2表格otc_2017Q3.csv
✅ 已更新並儲存：MOPS_第2表格otc_2017Q3.csv

📄 處理中：MOPS_第2表格otc_2017Q4.csv
✅ 已更新並儲存：MOPS_第2表格otc_2017Q4.csv

📄 處理中：MOPS_第2表格otc_2018Q1.csv
✅ 已更新並儲存：MOPS_第2表格otc_2018Q1.csv

📄 處理中：MOPS_第2表格otc_2018Q2.csv
✅ 已更新並儲存：MOPS_第2表格otc_2018Q2.csv

📄 處理中：MOPS_第2表格otc_2018Q3.csv
✅ 已更新並儲存：MOPS_第2表格otc_2018Q3.csv

📄 處理中：MOPS_第2表格otc_2018Q4.csv
✅ 已更新並儲存：MOPS_第2表格otc_2018Q4.csv

📄 處理中：MOPS_第2表格otc_2019Q1.csv
✅ 已更新並儲存：MOPS_第2表格otc_2019Q1.csv

📄 處理中：MOPS_第2表格otc_2019Q2.csv
✅ 已更新並儲存：MOPS_第2表格otc_2019Q2.csv

📄 處理中：MOPS_第2表格otc_2019Q3.csv
✅ 已更新並儲存：MOPS_第2表格otc_2019Q3.csv

📄 處理中：MOPS_第2表格otc_2019Q4.csv
✅ 已更新並儲存：

In [4]:
import pandas as pd
import os

def merge_all_tables(folder_path, output_name="MOPS_第2表格otc_總表.csv"):
    all_dfs = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".csv") and filename.startswith("MOPS_第2表格otc_"):
            file_path = os.path.join(folder_path, filename)
            try:
                df = pd.read_csv(file_path)
                all_dfs.append(df)
                print(f"✅ 已讀取：{filename}")
            except Exception as e:
                print(f"⚠ 讀取失敗：{filename} ➜ {e}")

    if not all_dfs:
        print("❌ 沒有可合併的檔案")
        return

    # 合併所有表格
    merged_df = pd.concat(all_dfs, ignore_index=True)

    # 拆解年季欄位為排序用
    merged_df["__年"] = merged_df["年季"].str[:4].astype(int)
    merged_df["__季"] = merged_df["年季"].str[-1].astype(int)

    # 排序後刪除臨時欄位
    merged_df = merged_df.sort_values(by=["__年", "__季"]).drop(columns=["__年", "__季"])

    # 存檔
    output_path = os.path.join(folder_path, output_name)
    merged_df.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"\n📦 已整合並儲存：{output_path}")

if __name__ == "__main__":
    folder = "./"  # 同一資料夾
    merge_all_tables(folder)


✅ 已讀取：MOPS_第2表格otc_2016Q1.csv
✅ 已讀取：MOPS_第2表格otc_2016Q2.csv
✅ 已讀取：MOPS_第2表格otc_2016Q3.csv
✅ 已讀取：MOPS_第2表格otc_2016Q4.csv
✅ 已讀取：MOPS_第2表格otc_2017Q1.csv
✅ 已讀取：MOPS_第2表格otc_2017Q2.csv
✅ 已讀取：MOPS_第2表格otc_2017Q3.csv
✅ 已讀取：MOPS_第2表格otc_2017Q4.csv
✅ 已讀取：MOPS_第2表格otc_2018Q1.csv
✅ 已讀取：MOPS_第2表格otc_2018Q2.csv
✅ 已讀取：MOPS_第2表格otc_2018Q3.csv
✅ 已讀取：MOPS_第2表格otc_2018Q4.csv
✅ 已讀取：MOPS_第2表格otc_2019Q1.csv
✅ 已讀取：MOPS_第2表格otc_2019Q2.csv
✅ 已讀取：MOPS_第2表格otc_2019Q3.csv
✅ 已讀取：MOPS_第2表格otc_2019Q4.csv
✅ 已讀取：MOPS_第2表格otc_2020Q1.csv
✅ 已讀取：MOPS_第2表格otc_2020Q2.csv
✅ 已讀取：MOPS_第2表格otc_2020Q3.csv
✅ 已讀取：MOPS_第2表格otc_2020Q4.csv
✅ 已讀取：MOPS_第2表格otc_2021Q1.csv
✅ 已讀取：MOPS_第2表格otc_2021Q2.csv
✅ 已讀取：MOPS_第2表格otc_2021Q3.csv
✅ 已讀取：MOPS_第2表格otc_2021Q4.csv
✅ 已讀取：MOPS_第2表格otc_2022Q1.csv
✅ 已讀取：MOPS_第2表格otc_2022Q2.csv
✅ 已讀取：MOPS_第2表格otc_2022Q3.csv
✅ 已讀取：MOPS_第2表格otc_2022Q4.csv
✅ 已讀取：MOPS_第2表格otc_2023Q1.csv
✅ 已讀取：MOPS_第2表格otc_2023Q2.csv
✅ 已讀取：MOPS_第2表格otc_2023Q3.csv
✅ 已讀取：MOPS_第2表格otc_2023Q4.csv
✅ 已讀取：MOPS_第2表格otc_2024Q1.csv
✅ 已讀取：MOPS